In [163]:
import pandas as pd
import numpy as np

# Electricity

In [191]:
d_e_nonscaled=pd.read_csv(r"EnergyPLAN files\Electricity\Hourly\el_demandBH_2020.txt",header=None,delimiter='\t',decimal=',')
d_eh_nonscaled=pd.read_csv(r"EnergyPLAN files\Heat\Hourly\demand_2020.txt",header=None,delimiter='\t',decimal=',')
d_ec_nonscaled=pd.read_csv(r"EnergyPLAN files\Heat\Hourly\CDD_BH_2020.txt",header=None,delimiter='\t',decimal=',')
d_transport_nonscaled=pd.read_csv(r"EnergyPLAN files\Transport\Hour_transport.txt",header=None,delimiter='\t',decimal=',')

In [192]:
d_e=d_e_nonscaled/d_e_nonscaled.sum(axis=0)
d_eh=d_eh_nonscaled/d_eh_nonscaled.sum(axis=0)
d_ec=d_ec_nonscaled/d_ec_nonscaled.sum(axis=0)
d_e_transport=d_transport_nonscaled/d_transport_nonscaled.sum(axis=0)

In [193]:
D_E=10.67#ann el demand TWh/ann
D_add=0.43
D_EH=2.9145+0.08/4.4 #HEAT PUMPS + EL HEATING
D_EC=0.2202
D_E_TRANSPORT=0.059

In [196]:
d_e_tot=D_E*d_e+D_add*d_e+D_E_TRANSPORT*d_e_transport-(D_EH*d_eh+D_EC*d_ec)
d_e_tot[d_e_tot[d_e_tot.columns[0]]<0]=0

In [197]:
d_e_tot.sum()

0    8.006118
dtype: float64

In [199]:
d_e_tot=d_e_tot.assign(Timestamp=pd.date_range("2020-01-01 00:00:00","2020-12-31 23:00:00", freq="1H"))
d_e_tot["Month"]=d_e_tot["Timestamp"].dt.month


In [200]:
(d_e_tot.groupby("Month")[0].mean())*1000000

Month
1      801.953595
2      829.401012
3      767.540468
4      812.632608
5      880.854279
6      991.665553
7     1011.064912
8      960.521048
9     1033.740844
10     991.081214
11     924.427906
12     930.907740
Name: 0, dtype: float64

# Heat_demand

## Individual heating

In [207]:
fuel_coal_domestic=1.15
fuel_oil_domestic=0.412#twh/ann
fuel_ngas_domestic=0.7125
fuel_biomas_domestic=13.47


q_coal_boiler=fuel_coal_domestic*0.8*d_eh
q_oil_boiler=fuel_oil_domestic*0.85*d_eh

q_ngas_boiler=fuel_ngas_domestic*0.9*d_eh
q_biomass_boiler=fuel_biomas_domestic*0.9*d_eh #0.9 is efficiency

In [208]:
print("Kotao na ugalj, domaćinstva",q_coal_boiler.sum(),"TWh/god")
print("Kotao na tečno gorivo, domaćinstva",q_oil_boiler.sum(),"TWh/god")
print("Kotao na prirodni gas, domaćinstva",q_ngas_boiler.sum(),"TWh/god")
print("Kotao na biomasu, domaćinstva",q_biomass_boiler.sum(),"TWh/god")

Kotao na ugalj, domaćinstva 0    0.92
dtype: float64 TWh/god
Kotao na tečno gorivo, domaćinstva 0    0.3502
dtype: float64 TWh/god
Kotao na prirodni gas, domaćinstva 0    0.64125
dtype: float64 TWh/god
Kotao na biomasu, domaćinstva 0    12.123
dtype: float64 TWh/god


## District heating

production_hourly=demand*ann production
demand=production*losses

In [233]:
production_dh=1.12916666666667 #TWh/god
loss_dh=0.3
production_chp=0.495833333333333 #TWh/god
loss_chp=0.3


In [234]:
production_dh_hourly=production_dh*d_eh
production_chp_hourly=production_chp*d_eh

In [235]:
demand_dh=production_dh*loss_dh*d_eh
demand_chp=production_chp*loss_chp*d_eh


In [236]:
demand_dh,demand_chp=demand_dh.assign(Timestamp=pd.date_range("2020-01-01 00:00:00","2020-12-31 23:00:00", freq="1H")),demand_chp.assign(Timestamp=pd.date_range("2020-01-01 00:00:00","2020-12-31 23:00:00", freq="1H"))
demand_dh["Month"],demand_chp["Month"]=demand_dh["Timestamp"].dt.month,demand_chp["Timestamp"].dt.month


production_dh_hourly,production_chp_hourly=production_dh_hourly.assign(Timestamp=pd.date_range("2020-01-01 00:00:00","2020-12-31 23:00:00", freq="1H")),production_chp_hourly.assign(Timestamp=pd.date_range("2020-01-01 00:00:00","2020-12-31 23:00:00", freq="1H"))
production_dh_hourly["Month"],production_chp_hourly["Month"]=production_dh_hourly["Timestamp"].dt.month,production_chp_hourly["Timestamp"].dt.month

In [237]:
(production_dh_hourly.groupby("Month")[0].mean())*1000000+(production_chp_hourly.groupby("Month")[0].mean())*1000000

Month
1     390.629616
2     306.640727
3     283.328209
4     190.173915
5     113.623502
6      70.461656
7      47.979097
8      40.552911
9      61.742908
10    146.804056
11    255.761816
12    314.874332
Name: 0, dtype: float64

In [238]:
(production_dh_hourly.groupby("Month")[0].mean())*1000000

Month
1     271.437503
2     213.075993
3     196.876781
4     132.146490
5      78.953767
6      48.961817
7      33.339321
8      28.179074
9      42.903405
10    102.009998
11    177.721672
12    218.797292
Name: 0, dtype: float64

In [239]:
(production_chp_hourly.groupby("Month")[0].mean())*1000000

Month
1     119.192114
2      93.564735
3      86.451428
4      58.027425
5      34.669735
6      21.499839
7      14.639776
8      12.373837
9      18.839503
10     44.794058
11     78.040144
12     96.077040
Name: 0, dtype: float64

In [240]:
demand_dh.sum()

0            0.33875
Month    57216.00000
dtype: float64

In [241]:
demand_chp.sum()

0            0.14875
Month    57216.00000
dtype: float64